In [2]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.LocalSession()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# S3 prefix
#bucket = sagemaker_session.default_bucket()
dependancy_path = "dependencies.py"
train_data = "s3://raghuassignment1/housing_train_data.csv"

In [3]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = "sklearn_entry_point_Latest.py"

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    dependencies=['dependencies.py'],
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="local",
    sagemaker_session=sagemaker_session,
)

In [4]:
sklearn_preprocessor.fit({"train": train_data})

Creating 3oae0apdve-algo-1-ugzcb ... 
Creating 3oae0apdve-algo-1-ugzcb ... done
Attaching to 3oae0apdve-algo-1-ugzcb
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:07,781 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:07,784 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:07,796 sagemaker_sklearn_container.training INFO     Invoking user training script.
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:07,973 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:07,992 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:08,007 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
3oae0apdve-algo-1-ugzcb | 2021-05-27 10:00:08,018 sagemaker-training-toolkit INFO     In

In [5]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, instance_type="local", assemble_with="Line", accept="text/csv"
)

In [6]:
# Preprocess training input
transformer.transform(train_data, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
#output_path="s3://empid-bucketname/trnasformer_train_data.csv"
transformer.wait()
preprocessed_train = transformer.output_path

Attaching to hzn31wpopp-algo-1-ku8aw
hzn31wpopp-algo-1-ku8aw | 2021-05-27 10:00:28,247 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
hzn31wpopp-algo-1-ku8aw | 2021-05-27 10:00:28,251 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
hzn31wpopp-algo-1-ku8aw | 2021-05-27 10:00:28,252 INFO - sagemaker-containers - nginx config: 
hzn31wpopp-algo-1-ku8aw | worker_processes auto;
hzn31wpopp-algo-1-ku8aw | daemon off;
hzn31wpopp-algo-1-ku8aw | pid /tmp/nginx.pid;
hzn31wpopp-algo-1-ku8aw | error_log  /dev/stderr;
hzn31wpopp-algo-1-ku8aw | 
hzn31wpopp-algo-1-ku8aw | worker_rlimit_nofile 4096;
hzn31wpopp-algo-1-ku8aw | 
hzn31wpopp-algo-1-ku8aw | events {
hzn31wpopp-algo-1-ku8aw |   worker_connections 2048;
hzn31wpopp-algo-1-ku8aw | }
hzn31wpopp-algo-1-ku8aw | 
hzn31wpopp-algo-1-ku8aw | http {
hzn31wpopp-algo-1-ku8aw |   include /etc/nginx/mime.types;
hzn31wpopp-algo-1-ku8aw |   default_type application/octet-stream;
hzn31wpopp-algo-1-ku8a

In [7]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()


model_name = "inference-" + timestamp_prefix
endpoint_name = "inference-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[scikit_learn_inferencee_model]
)

sm_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge", endpoint_name=endpoint_name)

---------------!

In [8]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, Predictor
#from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

csv_serializer = sagemaker.serializers.CSVSerializer(content_type="text/csv")
data = """-122.26,37.46,26,5067,750,1996,728,7.0001,NEAR OCEAN"""

sagemaker_session = sagemaker.Session()

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)


print(predictor.predict(data))

b'{"instances": [{"features": [-1.3408764930120565, 0.8514634482951091, -0.21099315585118414, 1.1430410231864847, 0.5257730939190086, 0.5164773116405509, 0.6146760713962672, 1.6402561279658028, 0.5819441198401629, -0.030616763948385764, -1.0052566158906469, 0.0, 0.0, 0.0, 0.0, 1.0]}]}'
